In [1]:
from huggingface_hub import InferenceClient


In [2]:
def read_token():
    with open(".token.txt", "r") as f:
        hf_token = f.read()
    return hf_token


    

In [3]:
hf_token = read_token()
client = InferenceClient("black-forest-labs/FLUX.1-dev", token=hf_token)

In [7]:
# output is a PIL.Image object
image = client.text_to_image("Astronaut riding a horse on moon")
image.show()

In [7]:
# output is a PIL.Image object
image = client.image_to_image(image="cruise.jpeg", prompt="Make the person run on a track", model="timbrooks/instruct-pix2pix")
image.show()

HfHubHTTPError: 504 Server Error: Gateway Timeout for url: https://api-inference.huggingface.co/models/timbrooks/instruct-pix2pix (Request ID: zzppyQoI1GeM5GpzaS8yG)

Model timbrooks/instruct-pix2pix time out

In [ ]:
import PIL
import requests
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler

model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)
pipe.to("cuda")
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

url = "https://raw.githubusercontent.com/timothybrooks/instruct-pix2pix/main/imgs/example.jpg"
def download_image(url):
    image = PIL.Image.open(requests.get(url, stream=True).raw)
    image = PIL.ImageOps.exif_transpose(image)
    image = image.convert("RGB")
    return image
image = download_image(url)

prompt = "turn him into cyborg"
images = pipe(prompt, image=image, num_inference_steps=10, image_guidance_scale=1).images
images[0]


In [12]:
def read_access_key():
    with open(".pico_access_key.txt", "r") as f:
        access_key = f.read()
    return access_key
pico_access_key = read_access_key()

In [11]:
import pvporcupine
from pvrecorder import PvRecorder
from datetime import datetime


keywords=['picovoice', 'bumblebee']

porcupine = pvporcupine.create(
    access_key=pico_access_key,
    keywords=keywords
)

recorder = PvRecorder(
    frame_length=porcupine.frame_length)

recorder.start()


def get_next_audio_frame():
    pass

try:
    while True:
        pcm = recorder.read()
        keyword_index = porcupine.process(pcm)
        
        if keyword_index >= 0:
            print(f"[{datetime.now()}] Detected {keywords[keyword_index]}")
    
except KeyboardInterrupt:
    print('Stopping ...')
finally:
    recorder.delete()
    porcupine.delete()
    
    

[2025-01-20 23:21:54.880386] Detected picovoice
[2025-01-20 23:22:03.932136] Detected bumblebee
Stopping ...


In [ ]:
import pvcheetah

cheetah = pvcheetah.create(access_key=pico_access_key)